In [1]:
import os
from huggingface_hub import login

hf_token = os.environ['HF_TOKEN']
login(hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
import torch
import transformers

model_id = "google/gemma-3-4b-it"
model_path = "./agricsense.model"

In [3]:
from datasets import load_dataset

DATASET_BASE = "/home/junhee0110/dataset/images_split_{}" #"/home/iamjunhee/OhMyData/SpatialQA/dataset/images_split_{}"
JSON_BASE = "NewQA_split_{}.json"
SPLIT_NUMBER = 1


get_dataset_dir = lambda index: DATASET_BASE.format(index)
get_json_dir = lambda index: os.path.join(DATASET_BASE.format(index), JSON_BASE.format(index))

dataset = load_dataset("json", data_files=get_json_dir(SPLIT_NUMBER), split="train")

get_image_path = lambda index, is_depth: os.path.join(get_dataset_dir(SPLIT_NUMBER), dataset[index]["image"][is_depth])

In [ ]:
from PIL import Image

Image.open(get_image_path(7779, 0))

In [5]:
from transformers import AutoProcessor, AutoModelForImageTextToText

model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
)

processor = AutoProcessor.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/junhee0110/.agricsense/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:550: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['language_model.lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [6]:
from pipeline import generate_prompt, collate_data

prompt = [
    generate_prompt("Which tree is the closest one?", get_image_path(0,0), get_image_path(0,1)),
    generate_prompt("Describe the image", get_image_path(0,0), get_image_path(0,1)),
    generate_prompt("How many trees do you see?", get_image_path(0,0), get_image_path(0,1)),
]

print(prompt)


[{'messages': [{'role': 'user', 'content': [{'type': 'image', 'path': '/home/junhee0110/dataset/images_split_1/visual_genome/3638.jpg'}, {'type': 'image', 'path': '/home/junhee0110/dataset/images_split_1/visual_genome_d/3638.png'}, {'type': 'text', 'text': 'Which tree is the closest one?'}]}]}, {'messages': [{'role': 'user', 'content': [{'type': 'image', 'path': '/home/junhee0110/dataset/images_split_1/visual_genome/3638.jpg'}, {'type': 'image', 'path': '/home/junhee0110/dataset/images_split_1/visual_genome_d/3638.png'}, {'type': 'text', 'text': 'Describe the image'}]}]}, {'messages': [{'role': 'user', 'content': [{'type': 'image', 'path': '/home/junhee0110/dataset/images_split_1/visual_genome/3638.jpg'}, {'type': 'image', 'path': '/home/junhee0110/dataset/images_split_1/visual_genome_d/3638.png'}, {'type': 'text', 'text': 'How many trees do you see?'}]}]}]


In [7]:
batch = collate_data(prompt, processor, for_generation=True).to(model.device, dtype=torch.bfloat16)

input_len = batch["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**batch, max_new_tokens=200, do_sample=False)
    generation = [result[input_len:] for result in generation]
decoded = [processor.decode(result, skip_special_tokens=True).strip("\n") for result in generation]


/home/junhee0110/.agricsense/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/junhee0110/.agricsense/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [8]:
print(decoded)

['The tree closest to the camera is the one on the right side of the image.', 'The image shows a body of water with trees on both sides. The trees on the left side are closer to the camera, while those on the right are further away. There is a fence running parallel to the water on both sides, and a building is visible in the background on the right side.', 'There are three trees in the image.']
